In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.layers as tfl  
from tensorflow.python.framework import ops 
import keras_tuner as kt

In [ ]:
#Normalize an array
def fn_normalize_col(x):
    if x.dtype == 'object' :
        return x
    return (x - np.mean(x))/np.std(x)

#Convert array to tensor and specify shape
def fn_target_to_tensor(data) :
    return tf.reshape(tf.convert_to_tensor(data),[data.shape[0],1])

#Split data into training and test sets using a defined probability
def fn_split_data(X,Y, train_prob = 0.8, seed = 0) : 
    
    X_train = X.sample(frac=train_prob, random_state=seed) 
    X_test   = X.drop(X_train.index) 
    Y_train = Y.loc[X_train.index] 
    Y_test = Y.drop(X_train.index) 
    
    X_train=tf.convert_to_tensor(X_train) 
    X_test=tf.convert_to_tensor(X_test)

    Y_train=fn_target_to_tensor(Y_train) 
    Y_test=fn_target_to_tensor(Y_test) 
    
    return [X_train,X_test,Y_train,Y_test] 
 
#Get and normalize data
def fn_get_coral_data(cols_X, cols_Y, cols_norm, train_prob = 0.9, seed = 0) : 
    
    data = pd.read_csv(
    "data//coral_total_cover.csv",header=0).dropna()
    
    X = data.copy().filter(cols_X)
    Y = data.copy().filter(cols_Y)
   
    Y = (Y > 8) * 1 
    X[["o_ocean"]] = (X[["o_ocean"]] == "CARIB" ) * 1
    
    X = X.apply(fn_normalize_col,axis=0)
    
    return fn_split_data(X,Y, train_prob, seed) 
 

In [ ]:
#Create hyperband tunable model
def tunable_model(hp) :
    
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    hp_l_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(11,)))
    model.add(tfl.Dense(hp_units, activation='relu'))  
    model.add(tfl.Dense(1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_l_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
               metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),  tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),])
    
    return model

In [ ]:
#Get training and test data
col_Y =  ["o_coral_cover"]
cols_X = ["o_ocean", "o_lat", "o_long", "o_depth",
           "o_human_pop50", "o_land_area50", "o_storm_30yr","o_l_att_mean", "o_npp_mean", "o_sst_mean", "o_wave_mean"]


cols_geo = ["o_ocean","o_lat", "o_long"]


cols_env = ["o_depth", "o_storm_30yr","o_l_att_mean", "o_npp_mean", "o_sst_mean", "o_wave_mean"]


cols_human = ["o_human_pop50", "o_land_area50"]

    
cols_sd  = ["o_lat", "o_long", "o_depth",
           "o_human_pop50", "o_land_area50", "o_storm_30yr","o_l_att_mean", "o_npp_mean", "o_sst_mean", "o_wave_mean"] 

input_size = len(cols_X)

X_train,X_test, Y_train,Y_test = fn_get_coral_data(cols_X,col_Y,cols_sd,train_prob=0.9, seed=230) 

In [ ]:
#Configure Tuner

tuner = kt.Hyperband(tunable_model,
                     objective='val_accuracy',
                     max_epochs=1000,
                     project_name='deepreef_SE',
                     factor=3)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [ ]:
# Get the optimal hyperparameters
tuner.search(X_train, Y_train, epochs=300, validation_split=0.1, callbacks=[stop_early])

best_hps=tuner.get_best_hyperparameters(num_trials=10)[0]

print(f"""optimal number of units is {best_hps.get('units')} and the optimal learning rate is {best_hps.get('learning_rate')}.
""")
 

In [ ]:
# Find Optimal Epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, Y_train, epochs=300, validation_split=0.1)

val_prec_per_epoch = np.array(history.history['val_precision'])
val_recall_per_epoch = np.array(history.history['val_recall'])

val_f1_per_epoch = (2 * ((val_prec_per_epoch * val_recall_per_epoch)/(val_prec_per_epoch + val_recall_per_epoch))).tolist()
best_epoch = val_f1_per_epoch.index(max(val_f1_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
#Get hyperband tuned model
hypermodel = tuner.hypermodel.build(best_hps)

# Train the model until best epoch
hypermodel.fit(X_train, Y_train, epochs=best_epoch)

In [ ]:
#Evaluate Test Set

eval_result = hypermodel.evaluate(X_test, Y_test)
print("[test loss, test accuracy, test f1]:", eval_result[0],  eval_result[1], 2*((eval_result[2]*eval_result[3])/(eval_result[2]+eval_result[3])))

In [ ]:
#Compute absolute mean of the input weights
abs(np.apply_along_axis(np.mean, 1,hypermodel.layers[0].get_weights()[0]))